In [3]:
import sys
sys.path.insert(0, '../')

import transformers
import torch.nn as nn
from transformers import AlbertModel, AlbertConfig
import torch
import argparse
import os
import easydict
from torch.utils.data import DataLoader, ConcatDataset
import pyxis.torch as pxt

from consonant.model.tokenization import NGRAMTokenizer
from consonant.model.modeling import Consonant


In [23]:
def load_tokenizer_model(ckpt):
    state = torch.load(ckpt)
    tokenizer = NGRAMTokenizer(state['ngram'])

    config = AlbertConfig(**state['config_dict'])
    model = Consonant(config)
    model.load_state_dict(state['model_state_dict'])
    return tokenizer, model

ckpt = '../ckpt-0027000.bin'
tokenizer, model = load_tokenizer_model(ckpt)


# Validation Set Inference

In [24]:
def val_dataloader(args):
        
    # We should filter out only directory name excluding all the *.tar.gz files
    data_dir = os.path.join(args.pretrain_dataset_dir, 'val') 
    subset_list = [subset_dir for subset_dir in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, subset_dir))]
    train_dataset = ConcatDataset([pxt.TorchDataset(os.path.join(data_dir, subset_dir)) for subset_dir in subset_list])

    # Very small dataset for debugging
    # toy_dataset = Subset(train_dataset, range(0, 100)) # -> If you want to make 100sample toy dataset. 

    data_loader = DataLoader(
        train_dataset,
        batch_size=args.train_batch_size,
        num_workers=args.num_workers,
        pin_memory=True,
        shuffle=False
    )

    return data_loader

In [25]:
args = easydict.EasyDict({
    "pretrain_dataset_dir": '../dataset/processed/ratings_3_100',
    "train_batch_size": 128,
    "num_workers": 0,
})

valloader = val_dataloader(args)
model.cuda()
model.eval()

Consonant(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(17579, 128, padding_idx=0)
      (position_embeddings): Embedding(100, 128)
      (token_type_embeddings): Embedding(1, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=512, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=512, out_features=512, bias=True)
                (key): Linear(in_features=512, out_features=512, bias=True)
                (value): Linear(in_features=512, out_features=512, bias=True)
          

In [26]:
with torch.no_grad():
    for i, batch in enumerate(valloader):
        input_ids = batch['head_ids'].type(torch.LongTensor).cuda()
        answer_label = batch['midtail_ids'].type(torch.LongTensor).cuda()  
        attention_mask = batch['attention_masks'].type(torch.LongTensor).cuda()  
        
        #print(input_ids.shape, attention_mask.shape,  answer_label.shape)
        output = model(input_ids, attention_mask=attention_mask, token_type_ids=None, answer_label=answer_label)

        if i > 2:
            break


In [27]:
predict_label = output[1].argmax(dim=2)

print('===============')
for i in range(answer_label.shape[0]):
    predict_label[i][answer_label[i]==0]=0
    answer_string = tokenizer.decode_sent(input_ids[i].detach().cpu().numpy(), answer_label[i].detach().cpu().numpy())
    predict_string = tokenizer.decode_sent(input_ids[i].detach().cpu().numpy(), predict_label[i].detach().cpu().numpy())
    #print('===============')
    print('answer string\t:'+ answer_string)
    print('predict string\t:' + predict_string)
    print('===============')
    

answer string	:되삭시다
predict string	:다시시다
answer string	:달콤한 인생. 이터널 선샤인 그리고 이 영화
predict string	:달콤한 인상. 인터넷 사상을 그렇게 왜 안해
answer string	:박수. 유역비고 뭐고 점정도는 된다!! 꼭봅시다!! 꼭 !다른사람 몇점주나 보자. 파이팅.
predict string	:박수. 알아보고 뭐가 저정도는 대다!! 꼴봅시다!! 꼭 !다른사람 만지주는 보자. 파이팅.
answer string	:베리굳.
predict string	:부로기.
answer string	:로버트 존쿠삭 존재감으로 본다
predict string	:로버트 지카스 자작곡으로 본다
answer string	:쓰레기 영화중에도 쓰레기 이게 뭐니??
predict string	:쓰레기 영화정에데 쓰레기 인간 뭐냐??
answer string	:조양은을 영웅화 시키는 어이없는영화...
predict string	:장은일을 영울한 시키는 어이이네야휴...
answer string	:정말 재미없네요...
predict string	:정말 재미있네요...
answer string	:초반부의 약간의 지루함을 감내 한다면 영화 보다 더 극적인 실화에 빠져들게 되리라. 평점.
predict string	:천벌받은 얼굴을 정리해여 그능 한다만 영화 보다 더 가정의 성활에 빠져들고 다려라. 표정.
answer string	:프랑스 영화의 자존심.
predict string	:프랑스 이해의 조작석.
answer string	:처음외엔 정말 볼게 없는 영화
predict string	:최음이요 정말 보고 없는 영화
answer string	:뭐... 이런영화가 다 있는지...
predict string	:뭐... 유런영화가 다 있니지...
answer string	:하 웃기시내 그리스가 머가 재밋다고.............
predict string	:헐 이기사는 그리세가 뭐가 잘못다고.............
answer st

# Single Sentence Inference

In [90]:
sentence = ["우리 오늘부터 커플 할까?"]

result = tokenizer.encode(sentence, max_char_length=50, return_attention_mask=True) #, return_tensors='pt')
head_ids = result['head_ids']
midtail_ids = result['midtail_ids']
attention_masks = result['attention_masks']

In [93]:
input_ids = torch.Tensor(head_ids).type(torch.LongTensor).cuda()
answer_label = torch.Tensor(midtail_ids).type(torch.LongTensor).cuda()  
attention_mask = torch.Tensor(attention_masks).type(torch.LongTensor).cuda()  

output = model(input_ids, attention_mask=attention_mask, token_type_ids=None, answer_label=answer_label)
predict_label = output[1].argmax(dim=2)

answer_string = tokenizer.decode_sent(input_ids[0].detach().cpu().numpy(), answer_label[0].detach().cpu().numpy())
predict_string = tokenizer.decode_sent(input_ids[0].detach().cpu().numpy(), predict_label[0].detach().cpu().numpy())

In [94]:
print(answer_string)
print(predict_string)

우리 오늘부터 커플 할까?
우래 오늘부터 커플 할까?
